
# Diseño de software para cómputo científico

----

## Unidad 1: Modelo de objetos de Python

<small><b>Source:</b> <a href="https://dbader.org/blog/python-dunder-methods">https://dbader.org/blog/python-dunder-methods</a></small>

### Agenda de la Unidad 1
---

- Clase 1:
    - Diferencias entre alto y bajo nivel.
    - Lenguajes dinámicos y estáticos.
    
- Limbo:
    - Introducción al lenguaje Python.
    - Librerías de cómputo científico.
    
- Clase Limbo + 1 y Limbo + 2:
    - **Orientación a objetos**, decoradores.

## Python Data types
-----

- Parte de la promesas incumplidas que tengo es explicar por que esto

In [1]:
[1, 2, 3] + [4, 5, 6]

[1, 2, 3, 4, 5, 6]

Funciona distinto de esto

In [2]:
import numpy as np

np.array([1, 2, 3]) + [4, 5, 6]

array([5, 7, 9])

## Python Data types
-----

al igual que estas dos cosas

In [3]:
np.array([1, 2, 3]) * 2

array([2, 4, 6])

In [4]:
[1, 2, 3] * 2

[1, 2, 3, 1, 2, 3]

## Python Data types
-----
O una de estas directamente no funciona

In [5]:
np.array([1, 2, 3]) + 1

array([2, 3, 4])

In [6]:
[1, 2, 3] + 1

TypeError: can only concatenate list (not "int") to list

### Dunders
----

- Por que funciona esto?

In [7]:
len([1, 3])

2

In [8]:
1 + 17

18

In [9]:
def foo():
    return "hello"

foo()  # <<< eso

'hello'

### Dunders
----

Vamos con el ejemplo siple

In [10]:
class HasLen:
    def __init__(self, l):
        self.l = l
    def __len__(self):
        return self.l

In [11]:
foo = HasLen(42)
len(foo)

42

In [12]:
foo.__len__()

42

### Dunders
----

- En Python, los métodos especiales son un conjunto de métodos predefinidos que se puede usar para enriquecer las clases. Son fáciles de reconocer porque comienzan y terminan con guiones bajos dobles, por ejemplo `__init__` o `__str__`.

- Como es cansador decir  *under-under-method-under-under* la comunidad empezo a decirles **dunder** contraccion de *double-under*.

- Los métodos Dunder le permiten emular el comportamiento de los tipos integrados. Por ejemplo, para obtener la longitud de una cadena puede llamar a `len('cadena')`.

### Dunders
----

Inicialización 

In [13]:
class Account:
    """A simple account class"""

    def __init__(self, owner, amount=0):
        """This is the constructor that lets us create
        objects from this class.
        
        """
        self.owner = owner
        self.amount = amount
        self._transactions = []
        
    def add_transaction(self, amount):
        if not isinstance(amount, int):
            raise ValueError('please use int for amount')
        self._transactions.append(amount)
        
    @property
    def balance(self):
        return self.amount + sum(self._transactions)

### Dunders
----

Representación

In [14]:
class Account(Account):

    def __repr__(self):
        return f'{self.__class__.__name__}({self.owner}, {self.amount})'

    def __str__(self):
        return f'Account of {self.owner} with starting amount: {self.amount}'

In [17]:
acc = Account('bob', 10)
acc  # repr(acc)

Account(bob, 10)

In [18]:
print(acc)  # str(acc)

Account of bob with starting amount: 10


### Dunders
----

Iteración

In [26]:
class Account(Account):

    def __len__(self):
        return len(self._transactions)

    def __getitem__(self, position):
        return self._transactions[position]
    
    def __setitem__(self, pos, v):
        self._transactions[pos] = v

In [27]:
acc = Account('bob', 10)
acc.add_transaction(20)
acc.add_transaction(-10)
acc.add_transaction(50)
acc.add_transaction(-20)
acc.add_transaction(30)

acc.balance

80

### Dunders
----

Iteración

In [28]:
len(acc)

5

In [32]:
for t in acc:
    print(t)

20
3
50
-20
30


In [31]:
acc[1]

3

### Dunders
----

Comparación

Los métodos son siempre con `__` al comienzo y al final: `ge, gt, le, lt, eq, ne`
Pero a partir de `__eq__` y algun otro se pueden completar automaticamente los demas con un decorador.

In [33]:
import functools

@functools.total_ordering
class Account(Account):
    # ... (see above)

    def __eq__(self, other):
        return self.balance == other.balance

    def __lt__(self, other):
        return self.balance < other.balance

In [34]:
acc2 = Account('tim', 100)
acc2.add_transaction(20)
acc2.add_transaction(40)
acc2.balance

160

### Dunders
----

Comparación

In [35]:
acc2 > acc

True

In [36]:
acc2 == acc

False

In [37]:
acc2 >= acc

True

### Dunders
----

Aritmética

Los métodos son siempre con `__` al comienzo y al final: `add, diff, mult, div, pow`, etc.

In [38]:
class Account(Account):
    
    def __add__(self, other):
        owner = f"{self.owner} + {other.owner}"
        start_amount = self.balance + other.balance
        return Account(owner, start_amount)

In [39]:
acc2 = Account("tim", 200)
acc2.balance

200

In [42]:
acc3 = acc2 + acc
acc3

Account(tim + bob, 293)

### Dunders
----

Objetos ejecutablles/llamables/callables

In [50]:
class Account(Account):
    # ... (see above)

    def __call__(self, c):
        print('Start amount: {}'.format(self.amount))
        print(c)
        print('Transactions: ')
        for transaction in self:
            print(transaction)
        print('\nBalance: {}'.format(self.balance))

In [51]:
acc2 = Account("tim", 200)
acc2.add_transaction(10)
acc2.balance

210

In [52]:
callable(acc2)

True

In [54]:
acc2("kk")

Start amount: 200
kk
Transactions: 
10

Balance: 210


Dunders Final
-------------

Hay muchos mas magic methods, pero eso se los dejo ver segun haga falta https://rszalski.github.io/magicmethods/

In [85]:
class A:
    def __init__(self, nombre):
        super().__setattr__("_data", {"nombre": nombre})
    def __getattr__(self, n):
        return self._data.get(n, "<UNK>")
    def __setattr__(self, n, v):
        self._data[n] = v
    def m(self):
        return 43  
        
a =A("tito")
a.nombresx = 1

In [79]:
import sh

In [84]:
sh.pwd()

/home/juan/proyectos/diseno_sci_sfw/src/unidad_1/mi_directorio

In [70]:
print(dir(a))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_data']
